Parte 2: Creación de Tabla Analítica

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.operations.gold_sales_daily_summary
USING DELTA 
AS 
SELECT 
--FECHA
f.BUSINESS_DATE,
YEAR(f.BUSINESS_DATE) AS YEAR,
MONTH(f.BUSINESS_DATE) AS MONTH,
DAY(f.BUSINESS_DATE) AS DAY, 
date_format(f.BUSINESS_DATE, 'EEEE') AS DAY_OF_WEEK,

--TIENDA
o.STORE_ID,
o.STORE_NAME, 
o.CITY,
o.REGION_NAME,
o.FORMAT_NAME,

--PRODUCTO
p.ITEM AS PRODUCT_CODE,
p.ITEM_DESC AS PRODUCT_DESCRIPTION, 
p.DEPT_NAME,
p.CLASS_NAME,
p.BRAND_NAME, 

--PROVEEDOR
s.SUP_NAME AS SUPPLIER_NAME, 

--CANAL
c.CHANNEL_NAME,
c.CHANNEL_TYPE_NAME, 

--MONEDA
curr.CURRENCY_ABR, 

--METRICAS
SUM(f.TOTAL_SALE) AS TOTAL_SALES,
SUM(f.TOTAL_SALE_TAX) AS TOTAL_SALES_TAX, 
SUM(f.TAX) AS TOTAL_TAX,
SUM(f.TOTAL_COST) AS TOTAL_COST, 
SUM(f.UNITS) AS UNITS_SOLD,
SUM(f.Q_TRANS) AS TOTAL_TRANSACTIONS,
--CALCULOS
(SUM(f.TOTAL_SALE)- SUM(f.TOTAL_COST)) AS GROSS_MARGIN,
((SUM(f.TOTAL_SALE) - SUM(f.TOTAL_COST)) / NULLIF(SUM(f.TOTAL_SALE), 0)) * 100 AS PERCENTAGE_GROSS_MARGIN, --Por si hay division entre cero, puse el nullif
(SUM(f.TOTAL_SALE_TAX) / NULLIF(SUM(f.Q_TRANS), 0)) AS AVERAGE_TICKET,
(SUM(f.UNITS) / NULLIF(SUM(f.Q_TRANS), 0)) AS UNITS_PER_TRANSACTION 

FROM workspace.operations.gold_fact_sales_product f 
INNER JOIN workspace.operations.gold_dim_organization o ON f.SK_ORGANIZATION = o.SK_ORGANIZATION
INNER JOIN workspace.operations.gold_dim_product p ON f.SK_PRODUCT = p.SK_PRODUCT
INNER JOIN workspace.operations.gold_dim_supplier s ON f.SK_SUPPLIER = s.SK_SUPPLIER
INNER JOIN workspace.operations.gold_dim_channel c on f.SK_CHANNEL = c.SK_CHANNEL
INNER JOIN workspace.operations.gold_dim_currency curr ON f.SK_CURRENCY = curr.SK_CURRENCY
GROUP BY f.BUSINESS_DATE, o.STORE_ID, o.STORE_NAME, o.CITY, o.REGION_NAME, o.FORMAT_NAME, p.ITEM, p.ITEM_DESC, p.DEPT_NAME, p.CLASS_NAME, p.BRAND_NAME, s.SUP_NAME, c.CHANNEL_NAME, c.CHANNEL_TYPE_NAME, curr.CURRENCY_ABR;



Parte 2.3 Validación de la Tabla Creada


In [0]:
%sql
--Verificación que se creo la tabla de gold sales daily summary correctamente
SELECT * FROM workspace.operations.gold_sales_daily_summary LIMIT 10;

In [0]:
%sql
--Número total de registros de la tabla creada
SELECT COUNT(*) AS total_registros 
FROM workspace.operations.gold_sales_daily_summary;

Hay 430910 registros.

In [0]:
%sql
--Validación que la suma de la venta sea la misma entre la tabla nueva y la tabla original
SELECT 
    (SELECT SUM(TOTAL_SALE) FROM workspace.operations.gold_fact_sales_product) AS venta_original,
    (SELECT SUM(TOTAL_SALES) FROM workspace.operations.gold_sales_daily_summary) AS venta_nueva,
    --Redondear los decimales
    ROUND((SELECT SUM(TOTAL_SALE) FROM workspace.operations.gold_fact_sales_product) - 
          (SELECT SUM(TOTAL_SALES) FROM workspace.operations.gold_sales_daily_summary), 2) AS diferencia;


In [0]:
%sql
--Validación que no haya valores nulos en campos críticos
SELECT 
SUM(CASE WHEN PRODUCT_CODE IS NULL THEN 1 ELSE 0 END) AS PRODUCTOS_NULOS,
SUM(CASE WHEN STORE_ID IS NULL THEN 1 ELSE 0 END) AS TIENDAS_NULAS,
SUM (CASE WHEN TOTAL_SALES IS NULL THEN 1 ELSE 0 END) AS VENTAS_NULAS
FROM workspace.operations.gold_sales_daily_summary;

Se confirmó que la tabla gold_sales_daily_summary mantiene la integridad financiera del modelo original, la diferencia entre las ventas fue 0 (hay una pequeña diferencia con los decimales pero es despreciable, ya que es del 0.000000000009%) y no hay valores nulos en dimensiones críticas. 